### SageMaker  Endpoint 部署ChatGLM
  
[ChatGLM](https://github.com/THUDM/ChatGLM-6B): ChatGLM-6B 是一个开源的、支持中英双语的对话语言模型，基于 General Language Model (GLM) 架构，具有 62 亿参数。结合模型量化技术，用户可以在消费级的显卡上进行本地部署（INT4 量化级别下最低只需 6GB 显存）。 ChatGLM-6B 使用了和 ChatGPT 相似的技术，针对中文问答和对话进行了优化。经过约 1T 标识符的中英双语训练，辅以监督微调、反馈自助、人类反馈强化学习等技术的加持，62 亿参数的 ChatGLM-6B 已经能生成相当符合人类偏好的回答。

#### 准备
1. 升级boto3, sagemaker python sdk  
2. 准备inference.py, requirements.txt

In [1]:
!pip install --upgrade boto3
!pip install --upgrade sagemaker

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 57.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.71
    Uninstalling botocore-1.29.71:
      Successfully uninstalled botocore-1.29.71
  Attempting uninstall: boto3
    Found existing installation: boto3 1.26.71
    Uninstalling boto3-1.26.71:
      Successfully uninstalled boto3-1.26.71
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.27.71 requires botocore==1.29.71, but you have botocore 1.29.118 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 743.3/743.3 kB 5.2 MB/s

In [2]:
import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

print(role)
print(bucket)

if "cn-" in region_name:
    with open('./code/requirements.txt', 'r') as original: data = original.read()
    with open('./code/requirements.txt', 'w') as modified: modified.write("-i https://pypi.tuna.tsinghua.edu.cn/simple\n" + data)

!touch dummy
!tar czvf model.tar.gz dummy
assets_dir = 's3://{0}/{1}/assets/'.format(bucket, 'chatglm')
model_data = 's3://{0}/{1}/assets/model.tar.gz'.format(bucket, 'chatglm')
!aws s3 cp model.tar.gz $assets_dir
!rm -f dummy model.tar.gz

model_name = None
entry_point = 'inference-chatglm.py'
framework_version = '1.13.1'
py_version = 'py39'
model_environment = {
    'SAGEMAKER_MODEL_SERVER_TIMEOUT':'600', 
    'SAGEMAKER_MODEL_SERVER_WORKERS': '1', 
}

from sagemaker.pytorch.model import PyTorchModel

model = PyTorchModel(
    name = model_name,
    model_data = model_data,
    entry_point = entry_point,
    source_dir = './code',
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
    env = model_environment
)

endpoint_name = 'pytorch-inference-chatglm-v1'
#instance_type = 'ml.p3.2xlarge'
instance_type='ml.g4dn.2xlarge' 

instance_count = 1

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type, 
    initial_instance_count = instance_count,
    serializer = JSONSerializer(),
    deserializer = JSONDeserializer()
)

arn:aws:iam::150333911459:role/NotebookStack-SmartSearchNotebookRole6F6BB12B-1TXWCZ1EG8JB9
sagemaker-ap-southeast-1-150333911459


### ChatGLM 测试


In [ ]:
#休眠2分钟,确保模型可以完全加载
import time
time.sleep(120)

In [8]:

inputs= {
    "ask": "你好!"

}

response = predictor.predict(inputs)
print(response["answer"])

inputs= {
    "ask": "晚上睡不着应该怎么办"

}

response = predictor.predict(inputs)
print(response["answer"])

你好👋！我是人工智能助手 ChatGLM-6B，很高兴见到你，欢迎问我任何问题。
以下是一些有助于在晚上入睡的方法：

1. 创建一个舒适的睡眠环境：保持房间安静、凉爽、黑暗，并使用舒适的床垫、枕头和床单。

2. 制定一个规律的睡眠时间表：每天在相同的时间上床和起床，有助于身体建立规律的睡眠习惯。

3. 避免刺激性物质：避免饮用含有咖啡因和酒精的饮料，并尽量减少摄入糖类和高脂肪的食物。

4. 放松身心：尝试进行一些放松的活动，如冥想、深呼吸或瑜伽，减轻压力和紧张情绪，更容易入睡。

5. 避免在床上使用电子设备：避免使用手机、平板电脑或电视等电子设备，这些设备会发出刺激性光线，干扰睡眠。

6. 尝试进行一些轻松的伸展运动：在床上进行一些轻松的伸展运动，让身体放松，有助于入睡。

如果仍然无法入睡，可以尝试咨询医生或专业医师，获取更专业的建议和帮助。


In [ ]:
inputs= {
    "ask": "列出一些年夜饭好意头的菜肴以及其寓意!"

}

response = predictor.predict(inputs)
print(response["answer"])

inputs= {
    "ask": "帮我写一篇人工智能课程的教案，1000字"

}

response = predictor.predict(inputs)
print(response["answer"])

In [ ]:
inputs= {
    "ask": "怎么修改huggingface transformers的model cache位置"

}

response = predictor.predict(inputs)
print(response["answer"])

inputs= {
    "ask": "用python3写出快速排序代码"

}

response = predictor.predict(inputs)
print(response["answer"])

In [ ]:
#我们来查看一下刚部署的这个ChatGLM模型对应的SageMaker inference endpoint
sagemaker_endpoint_name = predictor.endpoint_name
sagemaker_endpoint_name

利用已经在SageMaker real time inference endpoint部署的ChatGLM模型来模拟单轮对话和多轮对话。

下面的代码建议在SageMaker Notebook上来运行。

In [ ]:
import json
import boto3

client = boto3.client('runtime.sagemaker')

def query_endpoint_with_json_payload(encoded_json):
    response = client.invoke_endpoint(EndpointName=sagemaker_endpoint_name, ContentType='application/json', Body=encoded_json)
    return response

def parse_response_texts(query_response):
    model_predictions = json.loads(query_response['Body'].read())
    generated_text = model_predictions["answer"]
    return generated_text


先简单测试一下ChatGLM针对单个问题的回答

In [ ]:
payload = {"ask": "信息抽取：\n2022年世界杯的冠军是阿根廷队伍，梅西是MVP\n问题：国家名，人名\n答案："}
query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
generated_texts = parse_response_texts(query_response)
print(generated_texts)

单轮对话：每个问题/query都是独立的，问题之间没有关联性。

In [ ]:
#1.首先需要一个简单的开场拍。
print("用户：你好！\nChatGLM：您好!我是ChatGLM。我可以回答您的问题、写文章、写作业、翻译，对于一些法律等领域的问题我也可以给你提供信息。")

#2.在同一个session中持续对话，但是每次对话之间没有什么关联。
while True:
    command = input("用户：")
    if command == 'quit':
        break;
    
    #print(command)
    payload = {"ask": "\n用户："+ command + "\n"}
    #print(payload["ask"])
    query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
    generated_texts = "ChatGLM：" + parse_response_texts(query_response)
    print(generated_texts)

多轮对话模拟：我们这里来测试一下ChatGLM的多轮对话能力。

In [ ]:
#1.首先需要开场拍来引导ChatGLM，其实就是给它一个上下文来启动所谓的对话session。
payload = {"ask": "用户：你好！\nChatGLM：您好!我是ChatGLM。我可以回答您的问题、写文章、写作业、翻译，对于一些法律等领域的问题我也可以给你提供信息。"}
print(payload["ask"])
generated_texts = ""

#在这里简单模拟多轮对话时，发送给SageMaker endpoint的payload会越来越大，这里对payload大致做一个限制。
session_len = 10 * 1024 * 1024 

#2.在同一个session中持续对话，为了有多轮对话的效果，把每一轮的信息(问题-回答对)都带上来告诉ChatGLM之前的上下文。
while True:
    command = input("用户：")
    if command == 'quit':
        break;
    
    #print(command)
    whole_context = payload["ask"] + generated_texts + "\n用户："+ command + "\n"
    payload = {"ask": whole_context}
    if len(whole_context) > session_len:
        print("上下文信息太多了，当前对话session要退出了！")
        break;
    #print(payload["ask"])
    query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
    generated_texts = "ChatGLM：" + parse_response_texts(query_response)
    print(generated_texts)

### 删除SageMaker  Endpoint
删除推理服务

In [ ]:
predictor.delete_endpoint()